In [ ]:
!pip install sidpy SciFiReaders pycroscopy

In [1]:
colab = True #Set to True if in Google Colab, else set to False
if colab:
    %matplotlib widget
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    %matplotlib qt

# Creating a Reader

Here we are going to go through an example of how to create a pycroscopy reader

We will start with a stack of images of SrTiO3. The metadata is contained in a text file, and the images are contained in individual text files as well. We will need to read them in and then convert them to the sidpy dataset format. Luckily, this is not actually all that hard! Especially if we can leverage ChatGPT to figure out the tricky bits. 

The raw data is contained in the github repository here:
https://github.com/pycroscopy/arems25/tree/main/Data


In [2]:
#First let's import some packages
import json
import os
import re
import h5py
import sidpy as sid
import SciFiReaders as sr
import numpy as np


# Read the metadata

First, we need to read the metadata from the text file into a dictionary.

In [3]:
metadata_path = r'/Users/rvv/Github/arems25/Data/image_stack_metadata.txt'

# Convert the metadata text into a Python dictionary.
#Hint. Use json.read, and ChatGPT will be your friend.


# Read and arrange the image stack 

Here we are going to read the indivdiual image files that make up the stack and put them into a single numpy array.
HINT: You can use np.loadtxt(fname,delimiter=",") to load individual files.

The most challenging part of this exercise is to arrange the files in order. You can either try to do this yourself, or if you get tired, use an AI assistant.


In [4]:
#Now we have to import all of the text files

img_fnames = [fname for fname in os.listdir('Data') if 'img_' in fname]


# Create the sidpy Dataset

Once you have extracted the metadata and extracted the full data and placed it into a numpy arrray, we can begin to compile our sidpy dataset. For this, we need to construct our Dimension vectors, and then plug them into the set_dimension() method of sidpy's dataset class.

This page will be most helpful:

https://pycroscopy.github.io/sidpy/notebooks/00_basic_usage/create_dataset.html

In [6]:
#Now convert it to a sidpy dataset

#First, construct the x,y,and frame dimensions. And then construct the sidpy dataset. 


In [2]:
#Plot the data
#fig= data_set.plot()

# Bonus: Image Windowing

Now that we have the sidpy dataset, we can do some fancy things with it. For example, we can do atom finding, or image windowing, or atom statistics


In [16]:
#Image Windowing 

from pycroscopy.image import ImageWindowing

parms_dict = {}
parms_dict['window_step_x'] = 16
parms_dict['window_step_y'] = 16
parms_dict['window_size_x'] = 128
parms_dict['window_size_y'] = 128
parms_dict['mode'] = 'fft'
parms_dict['filter'] = 'hamming'
parms_dict['zoom_factor'] = 2
parms_dict['interpol_factor'] = 4
iw = ImageWindowing(parms_dict)
windows = iw.MakeWindows(data_set, dim_slice=0)
windows = np.abs(np.log(np.abs(windows)))


/Users/rvv/miniforge3/lib/python3.10/site-packages/pycroscopy/image/image_window.py:393: ComplexWarning: Casting complex values to real discards the imaginary part
  img_window = np.array(img_window, dtype = np.float32)


In [17]:
windows.plot();

In [18]:
from pycroscopy.learn.ml.matrix_factor import MatrixFactor

mfactor = MatrixFactor(np.abs(windows), method = 'nmf',n_components = 3)
output = mfactor.do_fit()

using generic parameters for dimension  1
using generic parameters for dimension  0


/Users/rvv/miniforge3/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/rvv/miniforge3/lib/python3.10/site-packages/sidpy/base/num_utils.py:54: RuntimeWarning: invalid value encountered in divide
  if var / step_avg < tol:


In [23]:
abundances = output[0]
components = output[1]
abund = np.array(abundances)
comps = np.array(components)

In [24]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(comps[ind,:,:])
    ax.set_title('Component #' + str(ind))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')
fig.tight_layout()
fig.savefig('Fig3b.png', dpi = 300)


fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(abund[:,:,ind])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')

fig.tight_layout()
fig.savefig('Fig3c.png', dpi = 300)